## 관련 환경설정 설치

In [ ]:
pip install -r requirements.txt

## Gemma 모델 준비

# 허깅 페이스에 로그인
from huggingface_hub import login

# Hugging Face API 토큰을 입력합니다.
api_token = "Your_Huggingface_Token"
login(api_token)

In [ ]:
# Gemma-2b-it 모델과 토크나이저 다운로드
import torch
import wandb

from sklearn.model_selection import train_test_split

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    Trainer
)
from transformers.intergrations import WandbCallback
from trl import DataCollatorForCompletionOnlyLM
import evaluate

# 모델과 토크나이저 불러오기
model_name = "googld/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(model_name,
    use_cache =False, # 빨리 불러오지만 메모리 사용량 증가
    device_map="auto",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    attn_implementation="eager",
)
tokenizer = AutoTokenizer.from_pretrained(model_name) # 해당 모델과 같이 학습시킨 토크나이저 로드

In [ ]:
# 데이터셋 준비
import datasets
dataset = datasets.load_dataset("jaehy12/news3")
print(dataset.keys())
element = dataset["train"][1] # train dataset의 1번째 data
element

In [ ]:
# 키워드 추출 기능 task 확인하기
input_text = """다음 텍스트를 한국어로 간단히 요약해주세요:\n부산의 한 왕복 2차선 도로에서 역주행 사고로 배달 오토바이 운전자인 고등학생이 숨지는 사고가 발생했다.
유족은 '가해자가 사고 후 곧바로 신고하지 않고 늑장 대응해 피해를 키웠다'고 주장하고 있다.
11일 부산진경찰서는 교통사고처리특례법(교통사고처리법)상 업무상 과실치사 혐의로 지난 3일 A(59)씨를 검찰에 불구속 송치했다고 밝혔다.
A씨는 교통사고처리법상 12대 중과실에 해당되는 '중앙선 침범'으로 역주행 교통사고를 일으킨 혐의를 받는다.
경찰에 따르면 스포츠유틸리티차량(SUV) 운전자 A씨는 5월 19일 밤 11시 50분쯤 부산진구 가야고가교 밑 도로에서 중앙선을 넘어 역주행으로 140m를 달려
반대편 차선의 오토바이 운전자 조모(16)군을 들이받았다. 조군은 원동기장치자전거 면허를 취득한 상태였고 헬멧도 쓰고 있었지만 크게 다쳤다.
사고 당일 수술을 받았으나 얼마 후 2차 뇌출혈로 뇌사 판정이 내려졌고, 사고 발생 약 한 달 만인 지난달 16일 끝내 사망했다.
사고를 낸 A씨는 술을 마시거나 약물을 복용한 상태에서 운전하지는 않은 것으로 조사됐다.
경찰 관계자는 'A씨가 자신이 정주행을 하고 오토바이가 역주행을 한 것으로 착각했다고 진술했다'고 설명했다."""

def change_inference_chat_format(input_text):
    return [
        # 입력문 요약을 위한 대화
        {"role": "user", "content": f"{input_text}"},
        {"role": "assistant", "content": """부산의 한 왕복 2차선 도로에서 역주행 사고로 배달
        오토바이 운전자인 고등학생이 숨지는 사고가 발생했다.
        유족은 '가해자가 사고 후 곧바로 신고하지 않고 늑장 대응해 피해를 키웠다'고 주장하고 있다.
        """},
        # 입력문 키워드 추출을 위한 대화
        {"role": "user", "content": "중요한 키워드 5개를 뽑아주세요."},
        {"role": "assistant", "content": ""}
    ]

prompt = change_inference_chat_format(input_text)

# prompt에 tokenizer 초기화 및 적용 
# 이로써 prompt의 text들은 토크나이저로 토큰단위로 분할되고 이는 vocab의 idx로 매핑
# 이는 모델이 텍스트를 해석할 수 있도록 vector로 바꿔주기 위함
inputs = tokenizer.apply_chat_template(prompt, tokenize=True,
add_generation_prompt=True, return_tensors="pt").to(model.device)

# 토크나이저로 벡터화된 input을 모델이 해석하여 답변 (outputs도 역시 벡터)
outputs = model.generate(inputs_ids=inputs.to(model.device), max_new_tokens=256)
# 벡터였던 outputs를 text화
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# 요약 기능 확인

# input_text : 같은 기사 활용
def change_inference_chat_format(input_text):
    return [
        {"role": "user", "content": f"{input_text}"},
        {"role": "assistant", "content": "한국어 요약:\n"}
    ]

# chat_template 적용
prompt = change_inference_chat_format(input_text)

# 생성
inputs = tokenizer.apply_chat_template(prompt, tokenize=True,
add_generation_prompt=True, return_tensors="pt").to(model.device)

outputs = model.generate(inputs, max_new_tokens=256, use_cache=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# 키워드 추출 & 요약 기능 확인

# input_text : 위에 정의된 기사와 동일
def change_inference_chat_format(input_text):
    return [
        {"role": "user", "content": f"다음 텍스트를 한국어로 간단히 요약하고,
            관련된 5개의 키워드를 추출해주세요:\n{input_text}"},
        {"role": "assistant", "content": ""},
    ]
prompt = change_inference_chat_format(input_text)

inputs = tokenizer.apply_chat_template(prompt, tokenize=True,
add_generation_prompt=True, return_tensors="pt").to(model.device)

outputs = model.generate(inputs, max_new_tokens=256, use_cache=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))